In [8]:
from lib.dataset.mesh_util import *
from lib.dataset.TestDataset import TestDataset

In [9]:
SMPLX_object = SMPLX()

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device

device(type='cuda')

Type of each parameter in 'dataset_param':

    <class 'str'>
    <class 'NoneType'>
    <class 'bool'>
    <class 'str'>
    <class 'int'>
    <class 'bool'>

In [11]:
dataset_param = {
        "image_dir": './examples/testimg/',
        "seg_dir": None,
        "use_seg": True,    # w/ or w/o segmentation
        "hps_type": 'pymafx' ,   # pymafx/pixie
        "vol_res": 256,  #cfg.vol_res,
        "single": False, #args.multi,
    }


In [12]:
dataset = TestDataset(dataset_param, device)

KeyError: 'seg_dir'